In [1]:
import numpy as np

In [24]:
import sys, os
sys.path.append('/Users/jenahii/DL study/.')
from deep_learning_from_scratch_master.common.util import im2col, col2im


In [3]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [4]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [5]:
array = np.array(
    
    [[[1 ,2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11,12,13,14,15],
    [16,17,18,19,20],
    [21,22,23,24,25]],
    
    [[1 ,2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11,12,13,14,15],
    [16,17,18,19,20],
    [21,22,23,24,25]]]
    )

array.shape

(2, 5, 5)

In [6]:
array = np.expand_dims(array, axis=0)
array.shape

(1, 2, 5, 5)

In [7]:
def im2col(array, filter_size:int, stride=1, pad = 0):

    padded_array = np.pad(array, [(0,0), (0,0), (pad,pad), (pad,pad)], mode='constant')

    filter_size = filter_size

    N, C, H, W = padded_array.shape

    out_h = (H  - filter_size)//stride + 1
    out_w = (W - filter_size)//stride + 1

    num_sliding_w = W - filter_size - (stride -1)
    num_slideing_h = H - filter_size - (stride -1)

    col_img = np.zeros((N, C, num_slideing_h, num_sliding_w, filter_size, filter_size))
    

    for y in range(filter_size):
        y_lim = y + out_h
        for x in range(filter_size):
            x_lim = x + out_w
            col_img[:,:,:,:,y,x] = padded_array[:,:,y:y_lim, x:x_lim]

    col_img = col_img.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)

    return col_img


In [8]:
x = np.random.rand(1,3,7,7)
col_x = im2col(x1, 5, stride=1, pad=0)
col_x.shape

(9, 75)

In [11]:
class Convolution:
    def __init__(self, W, b, stride = 1, pad=1):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

        self.x = None
        self.col = None
        self.col_w = None

        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 +(H +2*self.pad - FH)/self.stride)
        out_w = int(1+ (W + 2* self.pad - FW)/self.stride)

        col = im2col(x, FH, self.stride, self.pad)
        col_w = self.W.reshape(FN,  -1).T
        out = np.dot(col, col_w) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_w = col_w

        return out
    
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        
        dout = dout.transpose(0, 2, 3, 1).reshape(-1, FN)

        self.db = np.sum(dout, axis = 0)
        self.dxW = dout
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)
        dcol = np.dot(dout, self.col_w.T)
        dx = None

        return dx 

In [17]:
weight_x = np.random.rand(1, 2, 3, 3)
col_weight = weight_x.reshape(3, -1).T
col_weight.shape

(6, 3)

In [28]:
def col2im(col, input_shape, filter_size:int, stride=1, pad=0):
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_size)//stride + 1
    out_w = (W + 2*pad - filter_size)//stride + 1
    
    col = col.reshape(N, C, out_h, out_w, filter_size, filter_size).transpose(0, 3, 4, 5, 1, 2)
    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))

    print(img.shape) 

In [20]:
im_2 = col2im(col_x, (1,3,7,7), 5, stride=1)

(1, 3, 7, 7)


In [38]:
im = np.random.rand(10,3,64,64)
im_col = im2col(im, 3,3 , stride=2, pad=2)
im_col.shape

(10890, 27)

In [39]:
col_im = col2im(im_col, (10,3,64,64), 3, stride=2, pad=2)

(10, 3, 69, 69)


In [40]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride = 1, pad =0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride 
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_w = int(1+ ( H-self.pool_h)/self.stride)
        out_h = int(1 + (W - self.pool_w)/self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        out = np.max(col, axis=1)

        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

In [ ]:
class SimpleConvNet:
    def __init__(self, input_dim = (1, 28, 28),
                 conv_param = {'filter_num': 30, 'filter_size':5,
                               'pad':0, 'stride':1},
                hidden_size = 100, output_size = 10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride +1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        self.params ={}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads
    